# course 10 module 3

In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [3]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [44]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
print("max payload: ", max_payload)
print("min payload: ", min_payload)

max payload:  9600.0
min payload:  0.0


In [74]:
# Create a dash application
app = dash.Dash(__name__)

In [76]:
# Create an app layout
app.layout = html.Div(children = [html.H1('DASHBOARD SpaceX Launch Events and Outcomes',
                                          style={'textAlign': 'center', 'color': '#503D36',
                                                 'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Br(),
                                
                                dcc.Dropdown(id = 'dropdown_site',
                                             options = [
                                                       {'label': 'All Sites', 'value': 'ALL'},
                                                       {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                       {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                       {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                       {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                                       ],
                                             value = 'ALL',
                                             placeholder = 'Select Launch Site',
                                             searchable = True
                                             # style={'width':'80%','padding':'3px','font-size':'20px','text-align-last':'center'}
                                             ),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id = 'success_pie_chart')),
                                html.Br(),

                                html.P("Payload Range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                
                                dcc.RangeSlider(id = 'slider_payload',
                                                min = 0,
                                                max = 10000,
                                                step = 1000,
                                                value = [min_payload, max_payload]
                                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id = 'success_payload_scatter_chart')),
                                ])

In [78]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback(Output(component_id = 'success_pie_chart', component_property = 'figure'),
              Input(component_id = 'dropdown_site', component_property = 'value'))

def get_pie_chart(selected_site):
    filtered_df = spacex_df
    if selected_site == 'ALL':
        fig = px.pie(filtered_df, values = 'class', 
        names = 'Launch Site', 
        title = 'Success Count Across All Launch Sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        filtered_df = filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name = 'class count')
        fig = px.pie(filtered_df, values = 'class count', names = 'class', title = f"Success Count for Launch Site {selected_site}")
        return fig

In [80]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(Output(component_id = 'success_payload_scatter_chart', component_property = 'figure'),
                [Input(component_id = 'dropdown_site', component_property = 'value'),
                Input(component_id = 'slider_payload', component_property = 'value')])

def scatter(selected_site, payload):
    df_subset = spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0], payload[1])]
    
    if selected_site == 'ALL':
        fig = px.scatter(df_subset, x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version Category', title = 'Success Count Across All Launch Sites by Payload Mass and Booster Version')
        return fig
    else:
        fig = px.scatter(df_subset[df_subset['Launch Site'] == selected_site], x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version Category', title = f"Success Count for Launch Site {selected_site} by Payload Mass and Booster Version")
        return fig

In [82]:
# Run the app
if __name__ == '__main__':
    app.run(jupyter_mode = "external")
    #app.run(debug = True, port = 8050)  # port = 8050

Dash app running on http://127.0.0.1:8050/
